In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install sentencepiece
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git 
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 3.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tltogm_e
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tltogm_e
  Resolved https://github.com/huggingface/transformers.git to commit d924390d5b6e5a02c564b265efdc40808aa9f3b3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 13.4 MB/s eta 0:00:0

In [2]:
"""
import sys
MODULE_PATHS = ["../../modules"]
for module_path in MODULE_PATHS:
    if module_path not in sys.path:
        sys.path.append(module_path)
"""

import json


import torch
#from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import BitsAndBytesConfig


%load_ext autoreload
%autoreload 2


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [3]:
model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

In [4]:
from peft import TaskType

#model.gradient_checkpointing_enable()
#model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, config)

# print number of trainable parameters
model.print_trainable_parameters()

trainable params: 1769472 || all params: 249347328 || trainable%: 0.7096414524241463


In [5]:
# Verifying the datatypes.
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes:
        dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total += v
for k, v in dtypes.items():
    print(k, v, v / total)

torch.float32 249347328 1.0


In [6]:
with open("m1_ft_dataset.json", "rb") as reader:
    ft_dataset = json.load(reader)

In [7]:
# load in pandas dataframe
import pandas as pd
df = pd.DataFrame(ft_dataset)
df.head()

,question,answer
0,Question: Context: Rods (peak response at $\la...,"To find the threshold values for cone cells, w..."
1,Question: Context: Rods (peak response at $\la...,I would rate my confidence on the previous ans...
2,Question: What are some advantages of using AO...,One advantage of using acousto-optic deflector...
3,Question: What are some advantages of using AO...,"The answer to the question is ""D. larger range..."
4,Question: What are some advantages of using AO...,"Yes, there are multiple correct options that a..."


In [8]:
# create input column = context + followup question and label column = answer
df["input"] = df["question"]
df["label"] = df["answer"]

# drop columns
df = df.drop(columns=["question", "answer"])

# drop index
df = df.reset_index(drop=True)

df.head()

,input,label
0,Question: Context: Rods (peak response at $\la...,"To find the threshold values for cone cells, w..."
1,Question: Context: Rods (peak response at $\la...,I would rate my confidence on the previous ans...
2,Question: What are some advantages of using AO...,One advantage of using acousto-optic deflector...
3,Question: What are some advantages of using AO...,"The answer to the question is ""D. larger range..."
4,Question: What are some advantages of using AO...,"Yes, there are multiple correct options that a..."


In [9]:
# split into train and test
from datasets import Dataset

# transform to huggingface dataset
dataset = Dataset.from_pandas(df)

In [10]:
dataset[0]

{'input': 'Question: Context: Rods (peak response at $\\lambda=507 \\mathrm{~nm}$ ) and cones (peak response at $555 \\mathrm{~nm}$ ) are the photosensitive cells in human eye. Although rods are more sensitive, they cannot register colors (unlike cones).\n\nQuestion: Given that the sensitivity of cone cells is $1 / 220$ of the rod cells, find the threshold values for cone cells.\nAnswer: ',
 'label': 'To find the threshold values for cone cells, we need to know the threshold value for rod cells. The threshold value is the minimum amount of light required to activate the cells.\n\nSince rods are more sensitive, we can assume that their threshold value is lower than that of cones. The threshold value for rod cells is typically around $10^{-6}$ $\\mathrm{cd/m^2}$.\n\nUsing the sensitivity ratio provided in the question, we can calculate the threshold value for cone cells as follows:\n\nThreshold value for cone cells = Threshold value for rod cells / Sensitivity ratio\n\nThreshold value fo

In [11]:
#tokenizer.pad_token = tokenizer.eos_token
MAX_INPUT_LENGTH = 512

# tokenize input into input_ids and attention_mask
#dataset = dataset.map(lambda x: tokenizer(x["input"], padding="longest", max_length=MAX_INPUT_LENGTH, truncation=True), batched=True)
dataset = dataset.map(lambda x: tokenizer(x["input"], padding=False, max_length=MAX_INPUT_LENGTH, truncation=True), batched=False)

# tokenize labels into label 
def tokenize_labels(example):
    #example["label_ids"] = tokenizer(example["label"], padding="max_length", truncation=True, max_length=MAX_INPUT_LENGTH)["input_ids"]
    #example["labels"] = tokenizer(example["label"], padding="longest", max_length=MAX_INPUT_LENGTH, truncation=True)["input_ids"]
    example["labels"] = tokenizer(example["label"], padding=False, max_length=MAX_INPUT_LENGTH, truncation=True)["input_ids"]
    return example
dataset = dataset.map(tokenize_labels, batched=False)

# put -100 in the labels where the attention mask == pad_id
"""
def mask_labels(example):
    #example["label_ids"] = [-100 if label == tokenizer.pad_token_id else label for label in example["label_ids"]]
    example["labels"] = [-100 if label == tokenizer.pad_token_id else label for label in example["labels"]]
    return example


dataset = dataset.map(mask_labels)
"""
# drop columns
dataset = dataset.remove_columns("input")
dataset = dataset.remove_columns("label")

# set format
#dataset.set_format("torch", columns=["input_ids", "attention_mask", "label_ids"])
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


# split into train and test with 95/5 split
dataset = dataset.train_test_split(test_size=0.05)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


Map:   0%|          | 0/6572 [00:00<?, ? examples/s]

Map:   0%|          | 0/6572 [00:00<?, ? examples/s]

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6243
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 329
    })
})

In [13]:
dataset["train"][1]["labels"]

tensor([   37,  1657,    96,   346,   265,   614,    35,    53,   121,  8788,
            8, 15037,    16,    84,     8,     3,     4,    18,  2866, 11638,
         5792,   190,    46,  3735,  2992,  7501,    42,    72, 11273,    38,
           34,  9016,   190,   177,     7,    49,  6266,    13,     8,  3735,
            5,   100,  6986,   250,     8,   177,     7,    49,  1037,  8074,
            7,     8,  1364,   827,     3,     4,    18,  2866,     7,     6,
         3140,     8,  1146,   827,     3,     4,    18,  2866,     7,    12,
        19314,     8, 11638,     5,   100,    54,   991,    12,   768,    23,
         8717,     7,    16,  8668,    41,  5890,  4987,  1054,  3059,  5984,
           61,  1383,   213,     8,   177,     7,    49,  6266,  2385,  7353,
          120,    72, 13809,    42,  4126,    49,   145,    79,   700,    33,
            5,     1])

In [14]:
len(dataset["train"][1]["labels"])

112

# Before training

In [27]:

example_input = dataset["test"][1]["input_ids"].to("cuda")
example_label = dataset["test"][1]["labels"].to("cuda")

output = model.generate(
    input_ids=example_input.unsqueeze(0),
    max_new_tokens=200,
    temperature=0.9,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=100.0

)

print(tokenizer.decode(output[0], skip_special_tokens=True))

The main idea of the passage is about how the MIPS R10000 processor differs from previous MPS processor. The main idea behind this question is that the Mips R10000 uses two pointers, WrPtr and RdPptr, respectively to keep track of a fetched instruction in the instructions cache where he can be written. The main idea for these two pointers are: How do they work together? The main idea of what're their value changes is that the two points are used to maintain track on the next available location in the cache when a new instruction is emitted by the processor. The main concept here is that there are four parallel circuit boards working in parallel inside the processor which are designed to make it easier than most other systems to write instruction codes into the correct locations. The second key idea is that both pointers are used to keep track of the next available location while decoding an instruction. The first


In [21]:
from transformers import DataCollatorForSeq2Seq
training_args = TrainingArguments(
    #auto_find_batch_size=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-3,
    fp16=False,
    #fp16=True,
    save_total_limit=4,
    logging_steps=1,
    output_dir="./outputs",
    save_strategy='epoch',
    #optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.1,
)
"""
class ConversationTrainer(Trainer):    
    def compute_loss(self, model, inputs, return_outputs=False):
        assert return_outputs == False, "we did not implement return_outputs=True in RewardTrainer.compute_loss"
        
        print("inputs: ", inputs)
        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        print("labels: ", labels)
        loss, outputs = Trainer.compute_loss(self, model, inputs, return_outputs=True)
        print("loss: ", loss)
        print("outputs: ", outputs)
        wefwefw
        return loss

"""


        


# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=training_args,
    data_collator=data_collator
)



In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.223200
2,1.864500
3,2.320000
4,2.183800
5,2.087100
6,2.294300
7,2.361100
8,1.876700
9,1.887300
10,1.832700


TrainOutput(global_step=390, training_loss=2.064943794103769, metrics={'train_runtime': 1314.7527, 'train_samples_per_second': 4.748, 'train_steps_per_second': 0.297, 'total_flos': 3815991765172224.0, 'train_loss': 2.064943794103769, 'epoch': 1.0})

# After training